<a href="https://colab.research.google.com/github/saincoder404/NLP_Projects/blob/main/Sentiment_Analysis_LSTM_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Sentiment_Analysis_LSTM_Deep_Learning***

In [ ]:
!pip install kaggle

***Import libraries***

In [ ]:
import os
import json

import pandas as pd
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


***Data Collection through kaggle API***

In [ ]:
kaggle_dictionary = json.load(open('kaggle.json'))

In [ ]:
# kaggle_dictionary.keys()

In [ ]:
# setup kaggle credential as environment variable
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [ ]:
# unzip the dataset
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


***Load the dataset***

In [ ]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [ ]:
data.replace({'sentiment': {'positive' : 1, 'negative' : 0 }}, inplace=True)

In [ ]:
data['sentiment']

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [ ]:
data['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [ ]:
# split the data into train data and test data

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


***Data Preprocessing***

In [ ]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen = 200)

In [ ]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [ ]:
print(x_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [ ]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [ ]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [ ]:
print(y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


***LSTM --> Long Short term memory***

In [ ]:
# Build model
model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim = 120, input_length = 200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 120)          600000    
                                                                 
 lstm_3 (LSTM)               (None, 128)               127488    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 727617 (2.78 MB)
Trainable params: 727617 (2.78 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# compile the model
model.compile(optimizer='adam',loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# model training
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 [==============================] - 286s 497ms/step - loss: 0.3914 - accuracy: 0.8245 - val_loss: 0.3600 - val_accuracy: 0.8390
Epoch 2/5
500/500 [==============================] - 227s 454ms/step - loss: 0.2700 - accuracy: 0.8927 - val_loss: 0.4267 - val_accuracy: 0.8575
Epoch 3/5
500/500 [==============================] - 223s 446ms/step - loss: 0.2271 - accuracy: 0.9104 - val_loss: 0.2942 - val_accuracy: 0.8783
Epoch 4/5
500/500 [==============================] - 221s 441ms/step - loss: 0.2011 - accuracy: 0.9220 - val_loss: 0.3794 - val_accuracy: 0.8422
Epoch 5/5
500/500 [==============================] - 223s 447ms/step - loss: 0.1751 - accuracy: 0.9328 - val_loss: 0.3242 - val_accuracy: 0.8758


In [ ]:
# Model Evaluation
loss,accuracy = model.evaluate(x_test, y_test)

313/313 [==============================] - 24s 74ms/step - loss: 0.3121 - accuracy: 0.8813


In [ ]:
print(f'Test Loss : {loss}')
print(f'Test Accuracy : {accuracy}')

Test Loss : 0.31210049986839294
Test Accuracy : 0.8812999725341797


In [ ]:
# Build predictive system
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment


In [ ]:
# example usage
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 390ms/step
The sentiment of the review is: positive


In [ ]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 129ms/step
The sentiment of the review is: negative


In [ ]:
# example usage
new_review = "This movie was ok but not that good."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 237ms/step
The sentiment of the review is: negative
